In [1]:
conda list


Note: you may need to restart the kernel to use updated packages.


Traceback (most recent call last):
  File "C:\Users\sahil\anaconda3\Scripts\conda-script.py", line 11, in <module>
    from conda.cli import main
ModuleNotFoundError: No module named 'conda'


In [3]:
from transformers import PegasusTokenizer , PegasusForConditionalGeneration
from bs4 import BeautifulSoup
import requests

In [4]:
# 2. Setup Model
model_name = "human-centered-summarization/financial-summarization-pegasus"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)


Downloading:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.34k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44k [00:00<?, ?B/s]

In [5]:
type(tokenizer)

transformers.models.pegasus.tokenization_pegasus.PegasusTokenizer

In [6]:
##scraping data from url using bs4

url = "https://au.finance.yahoo.com/news/china-restricting-tesla-use-uncovers-a-significant-challenge-for-elon-musk-expert-161921664.html"
r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')
paragraphs = soup.find_all('p')

In [7]:
paragraphs[:10]

[<p class="Fz(14px) LineClamp(1,1.3em) M(0)">Reporting live from Parliament House, sign up to get our Budget Breakdown.</p>,
 <p>Renewed political tensions between the U.S. and China — which came to light this week as the Biden administration sat down with their Chinese counterparts for the first time to discuss a range of issues — could ensnarl vehicle maker Tesla (<a class="link rapid-noclick-resp" data-ylk="slk:TSLA" href="https://finance.yahoo.com/quote/TSLA?p=TSLA&amp;.tsrc=fin-srch">TSLA</a>), which has pushed successfully into China in recent years.</p>,
 <p>In fact, the heightened rhetoric between the two super economic superpowers may have already had blowback on Elon Musk's electric car company.</p>,
 <p>The Chinese government is restricting the use of Tesla vehicles by military staff and employees of vital state-owned companies, <a class="link rapid-noclick-resp" data-ylk="slk:The Wall Street Journal reported Friday" href="https://www.wsj.com/articles/china-to-restrict-tesla

In [10]:
#creating an article using the words from paragraphs list
text = [paragraph.text for paragraph in paragraphs]
words = ' '.join(text).split(' ')[:400]
ARTICLE = ' '.join(words)

In [31]:
ARTICLE

'Reporting live from Parliament House, sign up to get our Budget Breakdown. Renewed political tensions between the U.S. and China — which came to light this week as the Biden administration sat down with their Chinese counterparts for the first time to discuss a range of issues — could ensnarl vehicle maker Tesla (TSLA), which has pushed successfully into China in recent years. In fact, the heightened rhetoric between the two super economic superpowers may have already had blowback on Elon Musk\'s electric car company. The Chinese government is restricting the use of Tesla vehicles by military staff and employees of vital state-owned companies, The Wall Street Journal reported Friday. Chinese officials reportedly have concerns that Tesla\'s cars — outfitted with various data collecting capabilities — could serve as leakers of national security secrets. "I have been saying for months now that Tesla\'s level of integration of their business, of their research, of their sort of geo-tracki

In [39]:
#encoding the Article for text generation using transformers
input_ids = tokenizer.encode(ARTICLE, return_tensors='pt')
output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
summary = tokenizer.decode(output[0], skip_special_tokens=True)

In [40]:
summary

'Tensions between U.S. and China may have already had blowback. Tesla has been successful in China in recent years'

In [41]:
input_ids.shape


torch.Size([1, 492])

In [37]:
output[0]

tensor([    0, 47615,   116,   317,   475,   107,   283,   107,   111,  1224,
          218,   133,   506,   196,  6049,  3367,   107, 11997,   148,   174,
         1147,   115,  1224,   115,   909,   231,     1])

# Building a news and sentiment pipeline

In [59]:
monitored_tickers = ['ETH','DOGE','GME', 'TSLA', 'BTC']


## search and scrape stock news from yahoo finance

In [60]:
def search_for_stock_news_urls(ticker):
    search_url = "https://www.google.com/search?q=yahoo+finance+{}&tbm=nws".format(ticker)
    r = requests.get(search_url)
    soup = BeautifulSoup(r.text, 'html.parser')
    atags = soup.find_all('a')
    hrefs = [link['href'] for link in atags]
    return hrefs

In [61]:
raw_urls = {ticker:search_for_stock_news_urls(ticker) for ticker in monitored_tickers}
raw_urls


{'ETH': ['/?sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQOwgC',
  '/?output=search&ie=UTF-8&tbm=nws&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQPAgE',
  '/search?q=yahoo+finance+ETH&tbm=nws&ie=UTF-8&gbv=1&sei=AvybYIidKoHRrQGB16UQ',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&source=lnms&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUIBygA',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=shop&source=lnms&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUICSgC',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=isch&source=lnms&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUICigD',
  'https://maps.google.com/maps?q=yahoo+finance+ETH&um=1&ie=UTF-8&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUICygE',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=vid&source=lnms&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUIDCgF',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=bks&source=lnms&sa=X&ved=0ahUKEwiIpoCRwsTwAhWBaCsKHYFrCQIQ_AUIDSgG',
  '/advanced_search',
  '/search?q=yahoo+finance+ETH&ie=UTF-8&tbm=nws&source=lnt&tbs=qdr:h&sa=X

In [62]:
len(raw_urls['DOGE'])

43

In [63]:
import re


In [64]:
unwanted_key = ['terms','privacy', 'accounts', 'support', 'preferences', 'maps']

In [65]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls: 
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [66]:
cleaned_urls = {ticker:strip_unwanted_urls(raw_urls[ticker], unwanted_key) for ticker in monitored_tickers}
cleaned_urls

{'ETH': ['https://finance.yahoo.com/news/ethereum-high-fly-183347298.html',
  'https://uk.finance.yahoo.com/news/bitcoin-ethereum-blockchain-cryptocurrencies-decentralised-finance-what-is-de-fi-102623503.html',
  'https://finance.yahoo.com/news/ethereum-litecoin-ripple-xrp-daily-003743827.html',
  'https://finance.yahoo.com/video/ethereum-surpasses-4-000-amid-155027064.html',
  'https://finance.yahoo.com/news/bitcoin-vs-ethereum-crypto-better-003034363.html',
  'https://finance.yahoo.com/news/why-ethereum-rise-143638663.html',
  'https://finance.yahoo.com/news/bitcoin-ethereum-dogecoin-elon-musk-cryptocurrencies-prices-12-may-083700636.html',
  'https://finance.yahoo.com/news/crypto-analyst-who-nailed-ethereum-climb-says-10000-is-next-204306727.html',
  'https://finance.yahoo.com/video/ethereum-won-t-put-bitcoin-173325638.html',
  'https://finance.yahoo.com/news/two-very-real-reasons-ethereum-162354486.html'],
 'DOGE': ['https://finance.yahoo.com/news/dogecoin-major-price-increase-stil

## Search and scrape cleaned urls

In [67]:
def scrape_and_process(URLs):
    ARTICLES = []
    for url in URLs: 
        r = requests.get(url)
        soup = BeautifulSoup(r.text, 'html.parser')
        paragraphs = soup.find_all('p')
        text = [paragraph.text for paragraph in paragraphs]
        words = ' '.join(text).split(' ')[:350]
        ARTICLE = ' '.join(words)
        ARTICLES.append(ARTICLE)
    return ARTICLES

In [68]:
articles = {ticker:scrape_and_process(cleaned_urls[ticker]) for ticker in monitored_tickers}
articles

{'ETH': ['While up to recently, Bitcoin (BTC) and ETH would pretty much move in unison, the former topped already on April 14 and is currently trading a the same price level it reached February 21. Meanwhile, ETH is 71% higher than on April 14 and gained almost 115% (!) since that same February date. Thus the two largest cryptocurrencies by market cap are heavily diverging. After running in parallel for many years, such divergence does not make my life any easier as an analyst. It means going back to the drawing board as all I can do is “anticipate, monitor, and adjust accordingly.” the more significant correction (50-70%) I anticipated in ETH has been postponed. In my previous update, see here, I showed the monthly chart and how extremely overbought ETH had become on that time frame. But “overbought can always become more overbought” (the monthly RSI5 went from 98.7 then to 99.3 now), and, indeed, “in Bull markets upside surprises while downside disappoints.” I did not show the daily 

In [78]:
len(articles['GME'])

10

## Summarizing all articles now


In [79]:
def summarize(articles):
    summaries = []
    for article in articles:
        input_ids = tokenizer.encode(article, return_tensors='pt')
        output = model.generate(input_ids, max_length=55, num_beams=5, early_stopping=True)
        summary = tokenizer.decode(output[0], skip_special_tokens=True)
        summaries.append(summary)
    return summaries

In [80]:
summaries = {ticker:summarize(articles[ticker]) for ticker in monitored_tickers}
summaries

{'ETH': ['Ethereum’s 5th wave target zone has been reached. Daily EWP, RSI and Fibonacci-based target zone are shown.',
  'Decentralised finance, also known as DeFi, is booming. Decentralised finance, also known as DeFi, is booming',
  'Ethereum ends the week up by 33.07%, Litecoin up 44.30%.',
  'Ethereum has gained more than 436% this year. Bitcoin has fallen below $20,000 for the first time since December',
  'Second-largest crypto is up four-fold in 2021 vs. Bitcoin. Wedbush sees crypto boom playing out for next decade',
  'Ether’s daily trading volume jumped to $2.7 billion in 2021.',
  'Ethereum breaks $500bn market cap, up 4.7%. Tesla users vote on whether to use dogecoin',
  'Digital asset has already soared more than 400% this year. Kaspar says network to adopt more efficient proof of stake model',
  'Kevin Simpson, Founder and Chief Investment Officer at Capital Wealth Planning.',
  'Digital asset is up more than 360% in 2021 vs. Bitcoin’s 100% gain this year.'],
 'DOGE': ['D

In [82]:
articles['ETH'][5]

'The interest in Ethereum technology has never been higher. Ethereum is the second-largest public Blockchain network by both market capitalization and daily trading volume. It has already undergone remarkable development in its young history and continues to grow steadily. In 2020, an average of $231 million in ether was traded daily on spot exchanges. To start 2021, that number jumped to $2.7 billion. The interest has in part come from the acceleration of Decentralized Finance (Defi), a concept that focuses on blockchains and smart contracts rather than traditional financial intermediaries. It has also come from Ethereum layer 2 solutions – technology built on top of Ethereum that is highly scalable and efficient — and sizeable developments with stablecoins, which are mostly anchored on the Ethereum network. As such, the excitement and interest around Ether shows signs of growing acceptance and an appreciation of decentralization. Let’s look at three key reasons for the growing intere

## Applying sentiment analysis to the summaries

In [83]:
from transformers import pipeline
sentiment = pipeline('sentiment-analysis')

Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

In [94]:
summaries['DOGE'][9]

'Musk’s Saturday Night Live appearance failed to boost prices. Dogecoin earlier this week became the fourth-most valued coin'

In [95]:
sentiment(summaries['DOGE'][5])


[{'label': 'POSITIVE', 'score': 0.9952648878097534}]

In [98]:
scores = {ticker:sentiment(summaries[ticker]) for ticker in monitored_tickers}
scores

{'ETH': [{'label': 'POSITIVE', 'score': 0.8746117949485779},
  {'label': 'POSITIVE', 'score': 0.9995632171630859},
  {'label': 'NEGATIVE', 'score': 0.940664529800415},
  {'label': 'NEGATIVE', 'score': 0.9982873797416687},
  {'label': 'POSITIVE', 'score': 0.6359983086585999},
  {'label': 'NEGATIVE', 'score': 0.6294976472854614},
  {'label': 'NEGATIVE', 'score': 0.9836757779121399},
  {'label': 'POSITIVE', 'score': 0.9961393475532532},
  {'label': 'POSITIVE', 'score': 0.9861441850662231},
  {'label': 'POSITIVE', 'score': 0.7988760471343994}],
 'DOGE': [{'label': 'POSITIVE', 'score': 0.9960355758666992},
  {'label': 'NEGATIVE', 'score': 0.9977243542671204},
  {'label': 'NEGATIVE', 'score': 0.9915878176689148},
  {'label': 'NEGATIVE', 'score': 0.9994978308677673},
  {'label': 'NEGATIVE', 'score': 0.9996534585952759},
  {'label': 'POSITIVE', 'score': 0.9952648878097534},
  {'label': 'NEGATIVE', 'score': 0.9986428618431091},
  {'label': 'POSITIVE', 'score': 0.834967851638794},
  {'label': 'N

In [99]:
def create_output_array(summaries, scores, urls):
    output = []
    for ticker in monitored_tickers:
        for counter in range(len(summaries[ticker])):
            output_this = [
                ticker,
                summaries[ticker][counter],
                scores[ticker][counter]['label'],
                scores[ticker][counter]['score'],
                urls[ticker][counter]
            ]
            output.append(output_this)
    return output

In [100]:
final_output = create_output_array(summaries, scores, cleaned_urls)
final_output

[['ETH',
  'Ethereum’s 5th wave target zone has been reached. Daily EWP, RSI and Fibonacci-based target zone are shown.',
  'POSITIVE',
  0.8746117949485779,
  'https://finance.yahoo.com/news/ethereum-high-fly-183347298.html'],
 ['ETH',
  'Decentralised finance, also known as DeFi, is booming. Decentralised finance, also known as DeFi, is booming',
  'POSITIVE',
  0.9995632171630859,
  'https://uk.finance.yahoo.com/news/bitcoin-ethereum-blockchain-cryptocurrencies-decentralised-finance-what-is-de-fi-102623503.html'],
 ['ETH',
  'Ethereum ends the week up by 33.07%, Litecoin up 44.30%.',
  'NEGATIVE',
  0.940664529800415,
  'https://finance.yahoo.com/news/ethereum-litecoin-ripple-xrp-daily-003743827.html'],
 ['ETH',
  'Ethereum has gained more than 436% this year. Bitcoin has fallen below $20,000 for the first time since December',
  'NEGATIVE',
  0.9982873797416687,
  'https://finance.yahoo.com/video/ethereum-surpasses-4-000-amid-155027064.html'],
 ['ETH',
  'Second-largest crypto is u

In [101]:
import csv
with open('assetsummaries.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerows(final_output)